<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ASI_DEMO_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install openai -q
!pip install faiss-cpu -q
!pip install tiktoken -q
!pip install colab_env -q
!pip install --upgrade tiktoken -q

In [16]:
from IPython import get_ipython
from IPython.display import display



import openai
import faiss
import numpy as np
import tiktoken
import os
import colab_env


import colab_env

from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))
#client = OpenAI()

# Set your OpenAI API key
openai.api_key =  os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

def get_embedding(text, model="text-embedding-ada-002"):
    """
    Gets the embedding for the given text using the specified OpenAI model.
    """
    text = text.replace("\n", " ")  # Replace newlines, which can negatively affect performance.
    # Use client.embeddings.create instead of openai.Embedding.create
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

class Agent:
    """
    Represents an AI agent assisting with airline operations.
    """

    def __init__(self, name, cache_size, embedding_dim=1536):  # Use OpenAI's embedding dimension
        self.name = name
        self.cache_size = cache_size
        self.embedding_dim = embedding_dim
        self.index = faiss.IndexFlatL2(embedding_dim)
        self.knowledge_embeddings = []
        self.knowledge_data = []

    def load_knowledge(self, knowledge_source):
        """
        Loads knowledge into the Faiss index using OpenAI embeddings.
        """
        knowledge_data = {
            "maintenance issue": {
                "description": "Unexpected technical problems with the aircraft requiring inspection and repair.",
                "common_causes": ["mechanical failure", "avionics issues", "required inspections"],
                "impact": "Can cause significant delays as safety checks are paramount.",
                "mitigation": ["expedited repairs", "spare parts availability", "alternative aircraft"]
            },
            "weather": {
                "description": "Adverse weather conditions affecting flight operations.",
                "types": ["snow", "thunderstorms", "fog", "ice"],
                "impact": "Can cause delays, diversions, or cancellations depending on severity.",
                "mitigation": ["alternative routes", "de-icing procedures", "ground delays"]
            },
            # ... Add more entries for different delay reasons, airports, etc.
        }

        for key, value in knowledge_data.items():
            # Generate embedding using OpenAI's embedding model
            # The 'engine' argument is removed, as 'model' is already used for specifying the model.
            embedding = get_embedding(key, model='text-embedding-ada-002')
            self.knowledge_embeddings.append(embedding)
            self.knowledge_data.append(value)

        self.index.add(np.array(self.knowledge_embeddings))

    def retrieve_knowledge(self, query):
        """
        Retrieves information from the Faiss index based on a query, using OpenAI embeddings.
        """
        # Generate query embedding using OpenAI's embedding model
        # The 'engine' argument is removed, as 'model' is already used for specifying the model.
        query_embedding = get_embedding(query, model='text-embedding-ada-002')
        D, I = self.index.search(np.array([query_embedding]), k=3)  # Search for top 3 nearest neighbors

        relevant_info = []
        for i in I[0]:
            relevant_info.append(self.knowledge_data[i])
        return relevant_info

        for key, value in knowledge_data.items():
            # Generate embedding using OpenAI's embedding model
            embedding = get_embedding(key, engine='text-embedding-ada-002')
            self.knowledge_embeddings.append(embedding)
            self.knowledge_data.append(value)

        self.index.add(np.array(self.knowledge_embeddings))


    def handle_flight_delay(self, flight_number, departure_airport, delay_reason, delay_duration, weather_impact=None, airline_name="Air Canada"):
        """
        Handles a flight delay scenario.
        """
        # ... (rest of the code remains the same)

        # Retrieve relevant knowledge from the cache
        relevant_knowledge = self.retrieve_knowledge(delay_reason)
        relevant_knowledge += self.retrieve_knowledge(weather_impact)


        # Use the OpenAI API to generate solutions and communication
        # Changed from openai.ChatCompletion.create to client.chat.completions.create
        # Added max_tokens to control the output length
        # Included airline_name in the system message
        # Moved weather_info and knowledge_summary definition inside messages dictionary
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "system",
                "content": f"You are an AI agent assisting {airline_name} staff with flight delays. "
                           "Provide helpful solutions and draft communication to passengers using {airline_name}'s brand voice and guidelines."
            }, {
                "role": "user",
                # Defined weather_info and knowledge_summary here
                "content": f"Flight {flight_number} from {departure_airport} is delayed due to {delay_reason} for {delay_duration}. "
                          f"Weather impact: {weather_impact if weather_impact else ''} \nRelevant knowledge: {self.summarize_knowledge(relevant_knowledge)} "
                          "What should we do? How should we inform the passengers?"
            }],
            max_tokens=4096  # Increased max_tokens to allow for a longer response
        )

        print('\n')
        # Replace "[Your Name]" with "AI Agent 001"


        # Extract suggestions and communication from the response
        assistant_message = response.choices[0].message.content
        final_message = assistant_message.replace("[Your Name]", "AI Agent 001")
        #print(f"LLM response:\n{assistant_message}")
        print(f"LLM response:\n{final_message}")

    def summarize_knowledge(self, knowledge_list):
        """
        Summarizes the relevant knowledge for the LLM prompt.
        """
        if not knowledge_list:
            return ""
        summary = ""
        for item in knowledge_list:
            summary += f"Description: {item.get('description', 'N/A')}\n"
            summary += f"Impact: {item.get('impact', 'N/A')}\n"
            summary += f"Possible mitigation: {', '.join(item.get('mitigation', ['N/A']))}\n"
        return summary


# Create an agent
agent = Agent("FlightOpsAI", cache_size=1000)


# Load knowledge into the Faiss index
agent.load_knowledge("Airline Operations Database")

print('\n')
# Simulate a flight delay scenario with maintenance issue and Denver weather
agent.handle_flight_delay("AC1073", "Montreal", "maintenance issue", "2 hours", weather_impact="Heavy snow at Denver International Airport (DEN).")





LLM response:
In this situation, it's crucial to keep passengers informed about the reasons for the delay and provide updates on the estimated departure time. Here's a draft communication you can use to inform passengers about the delay:

Subject: Important Update Regarding Flight AC1073 from Montreal

Dear Valued Passengers,

We regret to inform you that Flight AC1073 from Montreal to Denver International Airport (DEN) is experiencing a delay of approximately 2 hours due to unexpected technical issues with the aircraft. Our maintenance team is working diligently to inspect and repair the aircraft to ensure your safety and comfort during the flight.

Furthermore, heavy snow at Denver International Airport is impacting our flight operations, which may lead to additional delays, diversions, or cancellations. Our team is closely monitoring the situation and exploring all possible mitigations, such as alternative routes, de-icing procedures, and ground delays, to minimize the inconveni